# Making a Database for Crime Reports Using Postgres

In this project, I use postgres to create a database for crime reports in Boston. 

In [59]:
import psycopg2

## Creating the Crime Database

In [60]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password='', host='localhost')

In [61]:
cur = conn.cursor()

In [62]:
conn.autocommit = True

In [63]:
cur.execute('CREATE DATABASE crime_db')

DuplicateDatabase: database "crime_db" already exists


In [ ]:
conn.autocommit = False

In [ ]:
conn.close()

## Creating a Crimes Table

In [106]:
conn = psycopg2.connect(dbname="crime_db", user="postgres", password='', host='localhost')

In [107]:
cur = conn.cursor()

In [108]:
cur.execute("CREATE SCHEMA crimes;")

### Choosing Appropriate Data Types for Each Column

In [109]:
import csv

In [110]:
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [111]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [112]:
first_row

['1',
 '619',
 'LARCENY ALL OTHERS',
 '2018-09-02',
 'Sunday',
 '42.35779134',
 '-71.13937053']

In [113]:
def get_col_set(csv_name, col_index):
    with open(csv_name,'r') as file:
        reader = csv.reader(file)
        next(reader)
        col_list = set()
        for row in reader:
            col_list.add(row[col_index])
        return col_list

In [114]:
get_col_set('boston.csv', 4)

{'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}

In [115]:
unique_val_dic = {}
for key, col in zip(col_headers, range(len(col_headers))):
    unique_val_dic[key] = len(get_col_set('boston.csv', col))

In [116]:
unique_val_dic

{'incident_number': 298329,
 'offense_code': 219,
 'description': 239,
 'date': 1177,
 'day_of_the_week': 7,
 'lat': 18177,
 'long': 18177}

In [117]:
description_max_length = 0 
for d in get_col_set('boston.csv', 2):
    if len(d) > description_max_length:
        description_max_length = len(d)

In [118]:
description_max_length

58

In [119]:
cur.execute("CREATE TYPE week_enum AS ENUM ('Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday');")

In [120]:
cur.execute('''CREATE TABLE crimes.boston_crimes (
incident_number INTEGER PRIMARY KEY,
offense_code INTEGER, 
description VARCHAR(100),
date DATE,
day_of_week week_enum,
lat DECIMAL,
long DECIMAL 

);''')

Above, we used an enumerated data type for the day of the week column because there are only seven possible options. We also used a varchar data type with a 100 character maximum for the description column, as all descriptions in the data were well under 100 characters. 

## Loading the Data into the Crimes Table

In [121]:
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

In [122]:
cur.execute("SELECT * FROM crimes.boston_crimes;")


In [123]:
print(len(cur.fetchall()))

298329


Above, we see that all the rows from the csv file have been added to the boston_crimes table. 

## Creating Appropriate Users and Permissions

In [124]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [125]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

In [126]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

In [127]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

In [128]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

In [129]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

In [130]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

In [131]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Testing Database Configuration

In [137]:
cur.execute('''SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin 
FROM pg_roles 
WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist', 'postgres');
''')
cur.fetchall()

[('postgres', True, True, True, True),
 ('readonly', False, False, False, False),
 ('readwrite', False, False, False, False),
 ('data_analyst', False, False, False, True),
 ('data_scientist', False, False, False, True)]

In [139]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
cur.fetchall()

[('readonly', 'SELECT'),
 ('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [140]:
conn.commit()

In [141]:
conn.close()